In [143]:
import numpy as np #import numpy library
import pandas as pd #import pandas library
import matplotlib.pyplot as plt #import matplot library
%matplotlib inline

import glob
import os

import nltk.classify.util
from astropy.table import Table, Column

import string

from sklearn import svm
from sklearn import metrics

In [144]:
 def loadMetricData(user, directory):
    path =  user + "/" + directory + "/"
#     print(path)
    all_files = glob.glob(os.path.join(path, "*.csv")) 
#     print(all_files)
    data = pd.DataFrame()
    list_ = []
    for file_ in all_files:
        data = pd.read_csv(file_,index_col=None, header=0)
        list_.append(data)
        data = pd.concat(list_)
    return data

In [145]:
# # clean times function
def cleanTime(df, col):
    df.col = pd.to_datetime(df.col)

In [146]:
metrics = loadMetricData('barbComp', "metrics")
metrics.tail(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp
1435,2016-05-02 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN
1436,2016-05-02 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN
1437,2016-05-02 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN
1438,2016-05-02 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN
1439,2016-05-02 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
metrics.dtypes

timestamp     object
heartrate    float64
steps        float64
calories     float64
gsr          float64
skintemp     float64
airtemp      float64
dtype: object

In [148]:
# clean metric data

metrics.timestamp = pd.to_datetime(metrics.timestamp)

metrics.heartrate = metrics.heartrate.fillna(0)
metrics.steps = metrics.steps.fillna(0)
metrics.calories = metrics.calories.fillna(0)
metrics.gsr = metrics.gsr.fillna(0)
metrics.skintemp = metrics.skintemp.fillna(0)
metrics.airtemp = metrics.airtemp.fillna(0)

# metrics.head(5)

In [149]:
# metrics = metrics[metrics.heartrate != 0]
# metrics.head(5)

In [150]:
activities = loadMetricData('barbComp', "activities")
# activities.tail(5)

In [151]:
sleep = loadMetricData('barbComp', "sleep")
# sleep.tail(5)

In [152]:
# sleep.dtypes

In [153]:
journal = loadMetricData('barbComp', "journal")
# journal.tail(5)

In [154]:
# journal.dtypes

In [155]:
# clean metric data
journal.starttime= pd.to_datetime(journal.starttime)
journal.endtime= pd.to_datetime(journal.endtime)

journal.qualitative = journal.qualitative.fillna(0)
journal.excited = journal.excited.fillna(0)
journal.happy = journal.happy.fillna(0)
journal.calm = journal.calm.fillna(0)
journal.anxious = journal.anxious.fillna(0)
journal.sad = journal.sad.fillna(0)
journal.angry = journal.angry.fillna(0)
journal.hungry = journal.hungry.fillna(0)
journal.tired = journal.tired.fillna(0)
journal.bored = journal.bored.fillna(0)
journal.ansaan = journal.ansaan.fillna(0)
journal.exhaca = journal.exhaca.fillna(0)

print(journal.shape)
journal.head(5)
# journal.tail(5)

(118, 24)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
0,2016-04-26 09:28:00,2016-04-26 09:33:00,4/26/16,9:28 AM,9:33 AM,"setting up basis, drinking coffee, listening t...",work,Calm,0,1,...,0,0,2,1,9,0,3,8,0,0
1,2016-04-26 09:33:00,2016-04-26 10:02:00,4/26/16,9:33 AM,10:02 AM,"listening to 80's pop, going through work email",work,Calm,0,0,...,0,0,2,0,8,0,2,8,0,0
2,2016-04-26 10:02:00,2016-04-26 10:25:00,4/26/16,10:02 AM,10:25 AM,meeting with student,work,Calm,0,0,...,0,0,4,0,6,0,0,6,0,0
3,2016-04-26 10:25:00,2016-04-26 10:35:00,4/26/16,10:25 AM,10:35 AM,meeting with mick,work,Calm,0,0,...,0,0,4,0,6,0,0,6,0,0
4,2016-04-26 10:35:00,2016-04-26 10:45:00,4/26/16,10:35 AM,10:45 AM,meeting with student,school,Calm,0,0,...,0,0,4,0,6,0,0,6,0,0


In [156]:

journal['boolexcited'] =  journal['excited'] > 0
journal['boolhappy'] =  journal['happy'] > 0
journal['boolcalm'] =  journal['calm'] > 0
journal['boolanxious'] =  journal['anxious'] > 0
journal['boolsad'] =  journal['sad'] > 0
journal['boolangry'] =  journal['angry'] > 0
journal['booltired'] =  journal['tired'] > 0
journal['boolhungry'] =  journal['hungry'] > 0
journal['boolbored'] =  journal['bored'] > 0
journal['exhaca'] =  journal['exhaca'] > 0
journal['ansaan'] =  journal['ansaan'] > 0

journal['exha'] =  journal['exha'] > 0
journal['haca'] =  journal['haca'] > 0
journal['anan'] =  journal['anan'] > 0
journal['ansa'] =  journal['anan'] > 0

    
boolexcited = journal.iloc[:, 18].values.reshape(journal.shape[0], 1)
boolhappy = journal.iloc[:, 19].values.reshape(journal.shape[0], 1)
boolcalm = journal.iloc[:, 20].values.reshape(journal.shape[0], 1)
boolanxious = journal.iloc[:, 21].values.reshape(journal.shape[0], 1)
boolsad = journal.iloc[:, 22].values.reshape(journal.shape[0], 1)
boolangry = journal.iloc[:, 23].values.reshape(journal.shape[0], 1)
booltired = journal.iloc[:, 24].values.reshape(journal.shape[0], 1)
boolhungry = journal.iloc[:, 25].values.reshape(journal.shape[0], 1)
boolbored = journal.iloc[:, 26].values.reshape(journal.shape[0], 1)
exhaca = journal.iloc[:, 27].values.reshape(journal.shape[0], 1)
ansaan = journal.iloc[:, 28].values.reshape(journal.shape[0], 1)
exha = journal.iloc[:, 29].values.reshape(journal.shape[0], 1)
haca = journal.iloc[:, 30].values.reshape(journal.shape[0], 1)
anan = journal.iloc[:, 31].values.reshape(journal.shape[0], 1)
ansa = journal.iloc[:, 32].values.reshape(journal.shape[0], 1)

journal.head(5)


,starttime,endtime,date,start,end,activity,location,generalemo,qualitative,excited,...,ansa,boolexcited,boolhappy,boolcalm,boolanxious,boolsad,boolangry,booltired,boolhungry,boolbored
0,2016-04-26 09:28:00,2016-04-26 09:33:00,4/26/16,9:28 AM,9:33 AM,"setting up basis, drinking coffee, listening t...",work,Calm,0,1,...,False,True,True,True,False,False,False,True,False,True
1,2016-04-26 09:33:00,2016-04-26 10:02:00,4/26/16,9:33 AM,10:02 AM,"listening to 80's pop, going through work email",work,Calm,0,0,...,False,False,True,True,False,False,False,True,False,False
2,2016-04-26 10:02:00,2016-04-26 10:25:00,4/26/16,10:02 AM,10:25 AM,meeting with student,work,Calm,0,0,...,False,False,False,True,False,False,False,True,False,False
3,2016-04-26 10:25:00,2016-04-26 10:35:00,4/26/16,10:25 AM,10:35 AM,meeting with mick,work,Calm,0,0,...,False,False,False,True,False,False,False,True,False,False
4,2016-04-26 10:35:00,2016-04-26 10:45:00,4/26/16,10:35 AM,10:45 AM,meeting with student,school,Calm,0,0,...,False,False,False,True,False,False,False,True,False,False


In [157]:
# for i in range(0, sarahJournal.shape[0]):
#     sarahJournal['ex'] = (0< sarahJournal.iloc[i, 18].astype(int))
#     sarahJournal['ha'] = sarahJournal.iloc[i, 19]== True
#     sarahJournal['ca'] = sarahJournal.iloc[i, 20]== True
#     sarahJournal['ys'] = sarahJournal['ex'].astype(int) + sarahJournal['ha'].astype(int) + sarahJournal['ca'].astype(int)

# print(sarahJournal['ex'])
    

In [158]:
# def makeEmoIndicators(newvar, journalData, journal_emo1_loc, journal_emo2_loc, journal_emo3_loc):
  
#     journalData[newvar] = False
    
#     for i in range(0, journalData.shape[0]):
#         journalData['emo1'] = (journalData.iloc[i, journal_emo1_loc]) == True)
#         journalData['emo2'] = (journalData.iloc[i, journal_emo2_loc] == True)
#         journalData['emo3'] = (journalData.iloc[i, journal_emo3_loc] == True)
#         journalData['ysum'] = journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int)
# #         print(journalData['emo1'].astype(int) + journalData['emo2'].astype(int) + journalData['emo3'].astype(int))
#         journalData.loc[journalData['ysum']!=0, newvar] = True
        
#     print(journalData['emo1'].values)
#     journalData = journalData.drop(['emo1', 'emo2', 'emo3', 'ysum'], axis=1)
#     return journalData

In [159]:
# sarahJournal.dtypes

In [160]:
# testData= makeEmoIndicators('exhaca', sarahJournal, 18, 19, 20)
# # print(testData.describe())

In [161]:
# sarahJournal.boolexhapcal.convert_objects(convert_numeric=True)

In [162]:
# function that takes any journal or activity data and creates indicators for activities or emotions

def makeIndicators(newvar, metricData, journalData, journal_startdatetime_loc, journal_enddatetime_loc, journal_activity_loc):
    '''
    Create boolean indicator variable for activities or emotions
    newvar, string for name of new indicator variable
    activity, string that matches the name of the activity or emotion in the journalData data frame
    bioData, data frame of input biometric data
    journalData, data frame of input journal or activity data
    journal_startdatetime_loc, int of position of the start datetime in journalData
    journal_enddatetime_loc, int of position of the end datetime in journalData
    journal_activity_loc, int of position of the activity or emotion in journalData
    '''

    metricData[newvar] = False

    for i in range(0, journalData.shape[0]):
        metricData['xlt'] = (metricData['timestamp'] >= pd.to_datetime(journalData.iloc[i, journal_startdatetime_loc]))
        metricData['xgt'] = (metricData['timestamp'] <= pd.to_datetime(journalData.iloc[i, journal_enddatetime_loc]))
#         metricData['xw'] = (journalData.iloc[i, journal_activity_loc])
        metricData['xw'] = (journalData.iloc[i, journal_activity_loc] == True)
        metricData['xsum'] = metricData['xlt'].astype(int) + metricData['xgt'].astype(int) + metricData['xw'].astype(int)
        metricData.loc[metricData['xsum']==3, newvar] = True
    
    metricData = metricData.drop(['xlt', 'xgt', 'xw', 'xsum'], axis=1)
    return metricData

In [163]:
# indicator for walking
totalData = makeIndicators('excitement', metrics, journal, 0, 1, 18)
totalData = makeIndicators('happy', metrics, journal, 0, 1, 19)
totalData = makeIndicators('calm', metrics, journal, 0, 1, 20)
totalData = makeIndicators('anxious', metrics, journal, 0, 1, 21)
totalData = makeIndicators('sad', metrics, journal, 0, 1, 22)
totalData = makeIndicators('angry', metrics, journal, 0, 1, 23)
totalData = makeIndicators('hungry', metrics, journal, 0, 1, 24)
totalData = makeIndicators('tired', metrics, journal, 0, 1, 25)
totalData = makeIndicators('bored', metrics, journal, 0, 1, 26)

totalData = makeIndicators('exhaca', metrics, journal, 0, 1, 27)
totalData = makeIndicators('ansaan', metrics, journal, 0, 1, 28)

totalData = makeIndicators('exha', metrics, journal, 0, 1, 29)
totalData = makeIndicators('haca', metrics, journal, 0, 1, 30)
totalData = makeIndicators('anan', metrics, journal, 0, 1, 31)
totalData = makeIndicators('ansa', metrics, journal, 0, 1, 32)




In [164]:
print(totalData.describe())

          heartrate         steps      calories           gsr      skintemp  \
count  10080.000000  10080.000000  10080.000000  10080.000000  10080.000000   
mean      65.284623      3.466567      1.658363      0.001082     70.396171   
std       35.596657     16.000542      1.467958      0.018148     35.013143   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%       57.000000      0.000000      1.100000      0.000055     82.400000   
50%       76.000000      0.000000      1.300000      0.000060     86.900000   
75%       88.000000      0.000000      1.900000      0.000073     88.700000   
max      198.000000    124.000000     17.200000      0.719000     97.700000   

            airtemp excitement     happy       calm   anxious     ...       \
count  10080.000000      10080     10080      10080     10080     ...        
mean      67.822421   0.509127   0.48006  0.0570437  0.509127     ...        
std       33.805046   0.499941  0.499627   0.231938  0

In [165]:
print(totalData.head())

            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-26 00:00:00          0      0         0    0         0        0   
1 2016-04-26 00:01:00          0      0         0    0         0        0   
2 2016-04-26 00:02:00          0      0         0    0         0        0   
3 2016-04-26 00:03:00          0      0         0    0         0        0   
4 2016-04-26 00:04:00          0      0         0    0         0        0   

  excitement  happy   calm  ...    angry hungry  tired  bored exhaca ansaan  \
0      False  False  False  ...    False  False  False  False  False  False   
1      False  False  False  ...    False  False  False  False  False  False   
2      False  False  False  ...    False  False  False  False  False  False   
3      False  False  False  ...    False  False  False  False  False  False   
4      False  False  False  ...    False  False  False  False  False  False   

    exha   haca   anan   ansa  
0  False  False  False  False 

In [166]:
totalData.to_csv('sarahTotal.csv')

In [167]:
totalData.head(5)

,timestamp,heartrate,steps,calories,gsr,skintemp,airtemp,excitement,happy,calm,...,angry,hungry,tired,bored,exhaca,ansaan,exha,haca,anan,ansa
0,2016-04-26 00:00:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2016-04-26 00:01:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,2016-04-26 00:02:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,2016-04-26 00:03:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,2016-04-26 00:04:00,0,0,0,0,0,0,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [168]:
# find day of week (0-6, Mon-Sun)
totalData['weekday'] = totalData['timestamp'].dt.weekday
totalData['month'] = totalData['timestamp'].dt.month
totalData['hour'] = totalData['timestamp'].dt.hour
totalData['min'] = totalData['timestamp'].dt.minute


print(totalData.shape)
print(totalData.head(5))

(10080, 26)
            timestamp  heartrate  steps  calories  gsr  skintemp  airtemp  \
0 2016-04-26 00:00:00          0      0         0    0         0        0   
1 2016-04-26 00:01:00          0      0         0    0         0        0   
2 2016-04-26 00:02:00          0      0         0    0         0        0   
3 2016-04-26 00:03:00          0      0         0    0         0        0   
4 2016-04-26 00:04:00          0      0         0    0         0        0   

  excitement  happy   calm ... exhaca ansaan   exha   haca   anan   ansa  \
0      False  False  False ...  False  False  False  False  False  False   
1      False  False  False ...  False  False  False  False  False  False   
2      False  False  False ...  False  False  False  False  False  False   
3      False  False  False ...  False  False  False  False  False  False   
4      False  False  False ...  False  False  False  False  False  False   

  weekday month hour min  
0       1     4    0   0  
1       1     

In [169]:
heartrate = totalData.iloc[:, 1].values.reshape(totalData.shape[0], 1)
steps = totalData.iloc[:, 2].values.reshape(totalData.shape[0], 1)
calories = totalData.iloc[:, 3].values.reshape(totalData.shape[0], 1)
gsr = totalData.iloc[:, 4].values.reshape(totalData.shape[0], 1)
skintemp = totalData.iloc[:, 5].values.reshape(totalData.shape[0], 1)
airtemp = totalData.iloc[:, 6].values.reshape(totalData.shape[0], 1)
# activity = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
# exercise = totalData.iloc[:, 17].values.reshape(totalData.shape[0], 1)
# thesis = totalData.iloc[:, 18].values.reshape(totalData.shape[0], 1)
# sally = totalData.iloc[:, 19].values.reshape(totalData.shape[0], 1)
# barb = totalData.iloc[:, 20].values.reshape(totalData.shape[0], 1)
# sol = totalData.iloc[:, 21].values.reshape(totalData.shape[0], 1)
# arash = totalData.iloc[:, 22].values.reshape(totalData.shape[0], 1)
# work = totalData.iloc[:, 23].values.reshape(totalData.shape[0], 1)
# home = totalData.iloc[:, 24].values.reshape(totalData.shape[0], 1)
# DayOfWeek = totalData.iloc[:, 25].values.reshape(totalData.shape[0], 1)
# Month = totalData.iloc[:, 26].values.reshape(totalData.shape[0], 1)
weekday = totalData.iloc[:, 7].values.reshape(totalData.shape[0], 1)
# month = totalData.iloc[:, 29].values.reshape(totalData.shape[0], 1)
hour = totalData.iloc[:, 8].values.reshape(totalData.shape[0], 1)
curmin = totalData.iloc[:, 9].values.reshape(totalData.shape[0], 1)

In [170]:
# concatenate to numpy dataset
X = np.concatenate((
        heartrate, 
        steps, 
        calories, 
        gsr, 
        skintemp, 
        airtemp, 
#         exercise, 
#         thesis, 
#         sally, 
#         barb, 
#         sol, 
#         arash, 
#         work, 
#         home, 
        weekday, 
        hour, 
        curmin
    ), axis=1)

X.shape

(10080, 9)

# Positive ( Excited, Happy, Calm)

In [171]:
# define y


y = totalData.exhaca.values

# y = totalData.excitement.values
y.shape

(10080,)

In [172]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [173]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [174]:
# report on training and test sets
global SVMerror, SVMacc, SVMtp, SVMtn, LRerror, LRacc, LRtp, LRtn, NBerror, NBacc, NBtp, NBtn, Perror, Pacc, Ptp, Ptn



def print_results(model):
    #print('Error rate on training set: ')
    erTRAIN = ((y_train != y_pred).sum() / X_train.shape[0])
    #print('Accuracy rate on training set: ')
    AccTRAIN = (1 - (y_train != y_pred).sum() / X_train.shape[0])
    #print('True positive rate on training tet:')
    TruPosTRAIN = ((y_train==True) & (y_pred==True)).sum() / y_train.sum()
    #TruNegTEST = (((y_train==False) & (y_pred_train==False)).sum() / (y_train.shape[0] - y_train.sum()))
    #print('**************')
    #('Error rate on test set: ')
    erTEST = ((y_test != y_pred_test).sum() / X_test.shape[0])
    #print('Accuracy rate on test set: ')
    AccTEST = (1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    #print('True positive rate on test set')
    TruPosTEST = (((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    #print('True negative rate on test set')
    TruNegTEST = (((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))
    data_rows = [('Error Rate', erTRAIN, erTEST),
                 ('Accuracy Rate', AccTRAIN, AccTEST),
                 ('True Positives', TruPosTRAIN, TruPosTEST),
                 ('True Negatives', '--', TruNegTEST)]
    t = Table(rows=data_rows, names=(model, 'Training Set', 'Test Set'), meta={'name': model + ': Training and Test Set Results'})
    print(t)
    if model == 'SVM':
        SVMerror = erTEST
        SVMacc = AccTEST
        SVMtp = TruPosTEST
        SVMtn = TruNegTEST
        return(SVMerror, SVMacc, SVMtp, SVMtn)
    elif model == 'Logistic Regression':
        LRerror = erTEST
        LRacc = AccTEST
        LRtp = TruPosTEST
        LRtn = TruNegTEST
        return(LRerror, LRacc, LRtp, LRtn)
    elif model == 'Naive Bayes':
        NBerror = erTEST
        NBacc = AccTEST
        NBtp = TruPosTEST
        NBtn = TruNegTEST
        return(NBerror, NBacc, NBtp, NBtn)
    elif model == 'Perceptron':
        Perror = erTEST
        Pacc = AccTEST
        Ptp = TruPosTEST
        Ptn = TruNegTEST
        return(Perror, Pacc, Ptp, Ptn)
    print('done')
      
    #t.show_in_browser(jsviewer=True) 
    
    
def all_models_table():
    all_rows = [('SVM', SVMerror, SVMacc, SVMtp, SVMtn),
            ('Logistic Regression', LRerror, LRacc, LRtp, LRtn),
            ('Naive Bayes', NBerror, NBacc, NBtp, NBtn),
            ('Perceptron', Perror, Pacc, Ptp, Ptn)]
    tt = Table(rows=all_rows, names=('', 'Error Rate', 'Accuracy', 'True +', 'True -'), meta={'3/15/2016'})
    print(tt)

## Positive ( Excited, Happy, Calm) Model Results

In [175]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM        Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.112103174603 0.104828042328
 Accuracy Rate 0.887896825397 0.895171957672
True Positives 0.938390611903    0.935546875
True Negatives             --         0.8745


(0.10482804232804233, 0.89517195767195767, 0.935546875, 0.87450000000000006)

In [176]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate  0.21556122449 0.216931216931
      Accuracy Rate  0.78443877551 0.783068783069
     True Positives 0.370913663034    0.361328125
     True Negatives             --          0.999


(0.21693121693121692, 0.78306878306878303, 0.361328125, 0.999)

In [177]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.157879818594 0.148148148148
 Accuracy Rate 0.842120181406 0.851851851852
True Positives 0.893964794635     0.91015625
True Negatives             --          0.822


(0.14814814814814814, 0.85185185185185186, 0.91015625, 0.82199999999999995)

In [178]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate  0.11862244898 0.114417989418
 Accuracy Rate  0.88137755102 0.885582010582
True Positives 0.952640402347      0.9453125
True Negatives             --          0.855


(0.11441798941798942, 0.88558201058201058, 0.9453125, 0.85499999999999998)

# Negative ( Anxious, Sad, Angry)

In [179]:
# define y
y = totalData.ansaan.values
y.shape

(10080,)

In [180]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [181]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Negative ( Anxious, Sad, Angry) Model Results

In [182]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0666099773243 0.0658068783069
 Accuracy Rate  0.933390022676  0.934193121693
True Positives             0.0             0.0
True Negatives              --  0.999646142958


(0.065806878306878314, 0.93419312169312163, 0.0, 0.99964614295824483)

In [183]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0666099773243 0.0658068783069
      Accuracy Rate  0.933390022676  0.934193121693
     True Positives             0.0             0.0
     True Negatives              --  0.999646142958


(0.065806878306878314, 0.93419312169312163, 0.0, 0.99964614295824483)

In [184]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0634920634921 0.0658068783069
 Accuracy Rate  0.936507936508  0.934193121693
True Positives  0.172340425532  0.161616161616
True Negatives              --  0.988322717622


(0.065806878306878314,
 0.93419312169312163,
 0.16161616161616163,
 0.98832271762208068)

In [185]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0666099773243 0.0654761904762
 Accuracy Rate  0.933390022676  0.934523809524
True Positives             0.0             0.0
True Negatives              --             1.0


(0.065476190476190479, 0.93452380952380953, 0.0, 1.0)

# Happy/Calm 

In [186]:
# define y
y = totalData.haca.values
y.shape

(10080,)

In [187]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [188]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Happy/Calm  Model Results

In [189]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0493197278912 0.0416666666667
 Accuracy Rate  0.950680272109  0.958333333333
True Positives             0.0             0.0
True Negatives              --             1.0


(0.041666666666666664, 0.95833333333333337, 0.0, 1.0)

In [190]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set      Test Set   
------------------- --------------- ---------------
         Error Rate 0.0493197278912 0.0416666666667
      Accuracy Rate  0.950680272109  0.958333333333
     True Positives             0.0             0.0
     True Negatives              --             1.0


(0.041666666666666664, 0.95833333333333337, 0.0, 1.0)

In [191]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0493197278912 0.0416666666667
 Accuracy Rate  0.950680272109  0.958333333333
True Positives             0.0             0.0
True Negatives              --             1.0


(0.041666666666666664, 0.95833333333333337, 0.0, 1.0)

In [192]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set     Test Set   
-------------- --------------- --------------
    Error Rate 0.0497448979592 0.042328042328
 Accuracy Rate  0.950255102041 0.957671957672
True Positives             0.0            0.0
True Negatives              -- 0.999309868875


(0.042328042328042326, 0.95767195767195767, 0.0, 0.99930986887508622)

# Anxious/Angry

In [193]:
# define y
y = totalData.anan.values
y.shape

(10080,)

In [194]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [195]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Angry Model Results

In [196]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00127551020408 0.000992063492063
 Accuracy Rate   0.998724489796    0.999007936508
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

In [197]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression   Training Set        Test Set    
------------------- ---------------- -----------------
         Error Rate 0.00127551020408 0.000992063492063
      Accuracy Rate   0.998724489796    0.999007936508
     True Positives              0.0               0.0
     True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

In [198]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set       Test Set    
-------------- ---------------- ----------------
    Error Rate 0.00198412698413 0.00198412698413
 Accuracy Rate   0.998015873016   0.998015873016
True Positives              0.0              0.0
True Negatives               --   0.999006951341


(0.001984126984126984, 0.99801587301587302, 0.0, 0.99900695134061568)

In [199]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron     Training Set        Test Set    
-------------- ---------------- -----------------
    Error Rate 0.00127551020408 0.000992063492063
 Accuracy Rate   0.998724489796    0.999007936508
True Positives              0.0               0.0
True Negatives               --               1.0


(0.00099206349206349201, 0.99900793650793651, 0.0, 1.0)

# Excited/Happy

In [200]:
# define y
y = totalData.exha.values
y.shape

(10080,)

In [201]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [202]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Excited/Happy Model Values

In [203]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM         Training Set     Test Set   
-------------- --------------- --------------
    Error Rate  0.253259637188 0.243386243386
 Accuracy Rate  0.746740362812 0.756613756614
True Positives 0.0315217391304     0.04296875
True Negatives              -- 0.999556737589


(0.24338624338624337, 0.75661375661375663, 0.04296875, 0.99955673758865249)

In [204]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression  Training Set     Test Set   
------------------- -------------- --------------
         Error Rate 0.163548752834 0.171626984127
      Accuracy Rate 0.836451247166 0.828373015873
     True Positives 0.970108695652     0.98046875
     True Negatives             -- 0.776595744681


(0.17162698412698413, 0.82837301587301582, 0.98046875, 0.77659574468085102)

In [205]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.106009070295  0.10582010582
 Accuracy Rate 0.893990929705  0.89417989418
True Positives 0.724456521739     0.72265625
True Negatives             -- 0.952570921986


(0.10582010582010581, 0.89417989417989419, 0.72265625, 0.95257092198581561)

In [206]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron    Training Set     Test Set   
-------------- -------------- --------------
    Error Rate 0.111394557823 0.112764550265
 Accuracy Rate 0.888605442177 0.887235449735
True Positives 0.770652173913 0.776041666667
True Negatives             -- 0.925088652482


(0.11276455026455026,
 0.88723544973544977,
 0.77604166666666663,
 0.92508865248226946)

# Anxious/Sad

In [207]:
# define y
y = totalData.ansa.values
y.shape

(10080,)

In [208]:
# create training and test sets
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=0)

In [209]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

## Anxious/Sad Model Results

In [210]:
# MODEL: SVM, linear
from sklearn import linear_model
clf = linear_model.SGDClassifier()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('SVM')

     SVM          Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000566893424036 0.000330687830688
 Accuracy Rate    0.999433106576    0.999669312169
True Positives               0.4               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [211]:
# MODEL: logistic regression
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='log', n_iter=50, alpha=0.00001)
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Logistic Regression')

Logistic Regression    Training Set        Test Set    
------------------- ----------------- -----------------
         Error Rate 0.000566893424036 0.000330687830688
      Accuracy Rate    0.999433106576    0.999669312169
     True Positives               0.4               0.0
     True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)

In [212]:
# MODEL: Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Naive Bayes')

 Naive Bayes     Training Set      Test Set   
-------------- --------------- ---------------
    Error Rate 0.0337301587302 0.0337301587302
 Accuracy Rate   0.96626984127   0.96626984127
True Positives             1.0             1.0
True Negatives              --  0.966258683427


(0.033730158730158728, 0.96626984126984128, 1.0, 0.96625868342705923)

In [213]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='perceptron')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results('Perceptron')

  Perceptron      Training Set        Test Set    
-------------- ----------------- -----------------
    Error Rate 0.000566893424036 0.000330687830688
 Accuracy Rate    0.999433106576    0.999669312169
True Positives               0.4               0.0
True Negatives                --               1.0


(0.00033068783068783067, 0.99966931216931221, 0.0, 1.0)